# MetaProgramming In Python

## Classes in Python - What is a class in Python?

In [82]:
class Test:
    pass


a = Test()
a

In [83]:
type(a)

__main__.Test

In [84]:
type(Test)

type

In [85]:
type(type)

type

### Classes - Nothing but instances of types.  Class technically is a sugar over the native 'type'

## What is type in Python?

In [86]:
?type

Init signature: type(self, /, *args, **kwargs)
Docstring:     
type(object) -> the object's type
type(name, bases, dict, **kwds) -> a new type
Type:           type
Subclasses:     ABCMeta, EnumType, _AnyMeta, NamedTupleMeta, _TypedDictMeta, _DeprecatedType, _ABC, MetaHasDescriptors, PyCStructType, UnionType, ...

In [87]:
TestWithType = type("TestWithType", (object,), {})

In [88]:
type(TestWithType)

type

In [89]:
ins1 = TestWithType()

In [90]:
type(ins1)

__main__.TestWithType

In [91]:
type("TestWithType", (object,), {})()

### 'type' is an important native structure used for creating classes.

## Life Cycle involved in a class - Vanilla

In [92]:
class TestClass:
    def __new__(cls, *args, **kwargs):
        print("new method called")
        instance = super(TestClass, cls).__new__(cls, *args, **kwargs)
        return instance

    def __call__(self, a, b, c):
        self.call_count += 1
        print("call method called")
        return a * b * c

    def __init__(self):
        self.call_count = 0
        super(TestClass, self).__init__()
        print("init method called")

    def get_call_count(self):
        return self.call_count

In [93]:
a = TestClass()

new method called
init method called


In [94]:
a(1, 2, 3)

call method called


6

In [95]:
a.get_call_count()

1

### What is type? 'type' defines how a class behaves in Python. 

### Got it. Well then - Can I change 'how' a class behaves in Python? - MetaClasses


## Metaclasses

In [96]:
class MySingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MySingletonMeta, cls).__call__(*args)
        return cls._instances[cls]

In [97]:
class MySingletonClass(metaclass=MySingletonMeta):
    def __init__(self):
        self.i = 1

In [98]:
a = MySingletonClass()
b = MySingletonClass()

In [99]:
type(a), id(a), type(b), id(b)

(__main__.MySingletonClass,
 1860914785936,
 __main__.MySingletonClass,
 1860914785936)

## LifeCycle with Metaclasses

In [100]:
class MyMetaClass(type):
    _test_attribute = 1

    def __new__(cls, *args, **kwargs):
        print("metaclass new method called")
        return super(MyMetaClass, cls).__new__(cls, *args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print("metaclass call method called")
        return super(MyMetaClass, cls).__call__(*args, **kwargs)

    def __init__(self, *args, **kwargs):
        print("metaclass init method called")
        return super(MyMetaClass, self).__init__(*args, **kwargs)

    def test_method_1(self):
        print("MyMetaClass - Test method 1 called")

In [101]:
class MyClass(metaclass=MyMetaClass):
    def __new__(cls, *args, **kwargs):
        print("instance new method called")
        return super(MyClass, cls).__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        print("instance init method called")
        return super(MyClass, self).__init__(*args, **kwargs)

metaclass new method called
metaclass init method called


In [102]:
ins2 = MyClass()

metaclass call method called
instance new method called
instance init method called


In [103]:
MyClass._test_attribute

1

In [104]:
MyClass.__mro__

(__main__.MyClass, object)

In [105]:
MyMetaClass.__mro__

(__main__.MyMetaClass, type, object)

## Pattern 1 : Abstract Classes

In [106]:
from abc import ABCMeta, ABC, abstractmethod

In [107]:
?ABCMeta

Init signature: ABCMeta(name, bases, namespace, /, **kwargs)
Docstring:     
Metaclass for defining Abstract Base Classes (ABCs).

Use this metaclass to create an ABC.  An ABC can be subclassed
directly, and then acts as a mix-in class.  You can also register
unrelated concrete classes (even built-in classes) and unrelated
ABCs as 'virtual subclasses' -- these and their descendants will
be considered subclasses of the registering ABC by the built-in
issubclass() function, but the registering ABC won't show up in
their MRO (Method Resolution Order) nor will method
implementations defined by the registering ABC be callable (not
even via super()).
File:           c:\program files\windowsapps\pythonsoftwarefoundation.python.3.11_3.11.752.0_x64__qbz5n2kfra8p0\lib\abc.py
Type:           type
Subclasses:     _ProtocolMeta, KernelProvisionerMeta, MySingletonABCMeta

In [108]:
class MyAbstractClass(metaclass=ABCMeta):
    def __init__(self):
        pass

    @abstractmethod
    def my_abstract_method(self):
        pass

In [109]:
try:
    MyAbstractClass()
except TypeError as ex:
    print(ex)

Can't instantiate abstract class MyAbstractClass with abstract method my_abstract_method


In [110]:
class MyChildClass(MyAbstractClass):
    def __init__(self):
        pass

    def my_abstract_method(self):
        pass

In [111]:
mcc = MyChildClass()
mcc

## Pattern 2 : Abstract family of singleton classes - Combine two metaclasses

In [112]:
class MySingletonABCMeta(ABCMeta):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MySingletonABCMeta, cls).__call__(*args)
        return cls._instances[cls]

In [113]:
class MyAbstractSingletonClass(metaclass=MySingletonABCMeta):
    def __init__(self):
        pass

    @abstractmethod
    def my_abstract_method(self):
        pass

In [114]:
try:
    MyAbstractSingletonClass()
except TypeError as ex:
    print(ex)

Can't instantiate abstract class MyAbstractSingletonClass with abstract method my_abstract_method


In [115]:
class MyAbstractSingletonChild(MyAbstractSingletonClass):
    def __init__(self):
        pass

    def my_abstract_method(self):
        pass

In [116]:
a1 = MyAbstractSingletonChild()
b1 = MyAbstractSingletonChild()

In [117]:
type(a1), id(a1), type(b1), id(b1)

(__main__.MyAbstractSingletonChild,
 1860902138768,
 __main__.MyAbstractSingletonChild,
 1860902138768)

## Pattern 3 : Pooled Objects

In [118]:
class MyBeanMeta(type):
    _instances = {}

    def __call__(cls, *args):
        print(args)
        key = tuple((cls, args))
        if key not in cls._instances:
            cls._instances[key] = super(MyBeanMeta, cls).__call__(*args)
        return cls._instances[key]

In [119]:
class MyBeanClass(metaclass=MyBeanMeta):
    def __init__(self, a):
        self.a = a

In [120]:
bn1 = MyBeanClass(1)
bn2 = MyBeanClass(2)
bn3 = MyBeanClass(3)
bn4 = MyBeanClass(1)

(1,)
(2,)
(3,)
(1,)


In [121]:
id(bn1), id(bn2), id(bn3), id(bn4)

(1860902009040, 1860902036432, 1860902140304, 1860902009040)

## Pattern 4 : Logging using Metaclasses

In [122]:
import logging

logging.basicConfig(filename="example.log", level=logging.INFO)
logging.debug("This message should go to the log file")
logging.info("So should this")
logging.warning("And this, too")


class MyLogSingletonMeta(type):
    logger = logging.getLogger("abc")

    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super(MyLogSingletonMeta, cls).__call__(*args)
            cls._instances[cls] = instance

            instance.__dict__["logger"] = logging.getLogger("abc")
        return cls._instances[cls]


class MyLogEnabledClass(metaclass=MyLogSingletonMeta):
    def test_function(self):
        self.logger.info("Inside test_function method of Log Enabled class")
        pass

In [123]:
lec_instance1 = MyLogEnabledClass()
lec_instance2 = MyLogEnabledClass()
lec_instance1.test_function()

print(id(lec_instance1), id(lec_instance2))

1860902048976 1860902048976


In [124]:
!cat example.log

INFO:root:So should this
INFO:abc:Inside test_function method of Log Enabled class
INFO:root:So should this
INFO:abc:Inside test_function method of Log Enabled class


In [125]:
class MyLogger:
    def __init__(self, logger=None):
        self.logger = logger

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            if self.logger is None:
                print(str(func) + " is called")
            else:
                self.logger.info(str(func) + " is called")
            return func(*args, **kwargs)

        return wrapper


class MyLoggingMeta(type):
    def __new__(cls, name, bases, attrs):
        for item, value in attrs.items():
            if callable(value):
                print("Function item :" + str(item), str(value), type(value))
                attrs[item] = MyLogger()(value)
            else:
                print(str(item), str(value), type(value))
        return super(MyLoggingMeta, cls).__new__(cls, name, bases, attrs)

In [126]:
class MyClass1(metaclass=MyLoggingMeta):
    def test_m1(self):
        pass

    def test_m2(self):
        pass

__module__ __main__ <class 'str'>
__qualname__ MyClass1 <class 'str'>
Function item :test_m1 <function MyClass1.test_m1 at 0x000001B1472E3240> <class 'function'>
Function item :test_m2 <function MyClass1.test_m2 at 0x000001B1472E2FC0> <class 'function'>


In [127]:
a = MyClass1()

In [128]:
a.test_m2()

<function MyClass1.test_m2 at 0x000001B1472E2FC0> is called


In [129]:
a.test_m1()

<function MyClass1.test_m1 at 0x000001B1472E3240> is called


## Pattern 5 : Sealed classes

In [130]:
class MySealedMeta(type):
    def __new__(cls, name, bases, attrs):
        all_metaclasses = [type(x) for x in bases]
        if MySealedMeta in all_metaclasses:
            raise TypeError("Sealed class cannot be sublcassed")
        return super(MySealedMeta, cls).__new__(cls, name, bases, attrs)

In [131]:
class MySealedClass(metaclass=MySealedMeta):
    pass

In [132]:
try:

    class MyChildOfSealed(MySealedClass):
        pass

except TypeError as ex:
    print(ex)

Sealed class cannot be sublcassed
